In [1]:
""" The main script for module RNN
"""
import os
import time
import math
import argparse
import pickle

import numpy as np
import tensorflow as tf
# from tensorflow.contrib.framework import nest

from utils import DataLoader, draw_strokes
from model import Model

/home/mye/tools/anacondas3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/mye/tools/anacondas3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [3]:
def train(args):
  data_loader = DataLoader(args.batch_size, args.data_scale, args.bptt_length) # args.bptt_length?????????
  data_loader.reset_batch_pointer()

  # model needs to know the dim of one-hot vectors
  args.char_vec_dim = data_loader.char_vec_dim
  # also the max length of char vector
  args.max_char_len = data_loader.max_char_len

  if args.model_dir != '' and not os.path.exists(args.model_dir):
    os.makedirs(args.model_dir)
  
  print(args.model_dir) ######!!!!!!!!!!!!!!!!
  with open(os.path.join(args.model_dir, 'config.pkl'), 'wb') as f:
    pickle.dump(args, f)
  print("hyperparam. saved.")

  model = Model(args)

  # training
  with tf.Session(config = config) as sess:

    tf.global_variables_initializer().run()

    saver = tf.train.Saver()
    
    # args.load_model: Reload a model checkpoint and restore training
    if args.load_model is not None:
        saver.restore(sess, args.load_model)
        print('args.load_model: ',args.load_model) #################!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        _, ep_start = args.load_model.rsplit("-", 1)
        ep_start = int(ep_start) # the last element of load_model
        model_steps = int(ep_start * data_loader.num_batches)
    else:
        ep_start = 0
        model_steps = last_model_steps = 0  # model_steps?????????????

    last_time = time.time()

    for ep in range(ep_start, args.num_epochs):
      ep_loss = []
      sess.run(tf.assign(model.lr, args.learning_rate * (args.decay_rate ** ep))) #????????
      print('data_loader.num_sequences: %d, args.batch_size: %d' %(data_loader.num_sequences, args.batch_size)) ###!!!
      for i in range(int(data_loader.num_sequences / args.batch_size)):
        idx = ep * data_loader.num_sequences + i * args.batch_size
        start = time.time()
        x, y, w, c = data_loader.next_batch()

        loss_list, model_steps = model.train(
          sess=sess, 
          sequence=x, 
          targets=y, 
          weights=w, 
          texts=c, 
          subseq_length=args.bptt_length, 
          step_count=model_steps
          ) # def train(self, sess, sequence, targets, weights, texts, subseq_length, step_count)

        ep_loss += loss_list

        if model_steps - last_model_steps >= 100:
          last_model_steps = model_steps
          new_time = time.time()
          print(
            "Sequence %d/%d (epoch %d), batch %d, train_loss = %.3f, time/(100*batch) = %.3f" 
            % (
                idx,
                args.num_epochs * data_loader.num_sequences,
                ep,
                model_steps,
                np.mean(loss_list),
                new_time - last_time
              ),
            flush=True
            )
          last_time = new_time
      print('x: ' ,type(x) , '; c', type(c) ) ####!!!
      print("Epoch %d completed, average train loss %.6f" % (ep, np.mean(ep_loss)))
      if not os.path.isdir(args.model_dir):
        os.makedirs(args.model_dir)
      if (ep+1) % args.save_every == 0:
        checkpoint_path = os.path.join(args.model_dir, 'model.ckpt')
        saver.save(sess, save_path=checkpoint_path, global_step = (ep+1))
        print("model saved.")

In [13]:
print(args.model_dir )
str = 'a-b c- d'
a = str.rsplit("-", 1)
print(a)
print(os.path.isdir(args.model_dir))

checkpoints
['a-b c', ' d']
True


### 1. Main func to train

In [2]:
def main():
  parser = argparse.ArgumentParser()
  parser.add_argument('--dim_rec', type=int, default=400,
                     help='size of RNN hidden state')
  parser.add_argument('--num_layers', type=int, default=3,
                     help='number of layers in the RNN. ' \
                     'Needs to be larger than 3 for synthesis.')
  parser.add_argument('--batch_size', type=int, default=50,
                     help='minibatch size')
  parser.add_argument('--num_epochs', type=int, default=200,
                     help='number of epochs')
  parser.add_argument('--save_every', type=int, default=5,
                     help='save frequency by epoches')
  parser.add_argument('--model_dir', type=str, default='checkpoints',
                     help='directory to save model to')
  parser.add_argument('--summary_dir', type=str, default='summary',
                     help='directory to save tensorboard info')
  parser.add_argument('--max_grad_norm', type=float, default=10.,
                     help='clip gradients at this value')
  parser.add_argument('--learning_rate', type=float, default=0.001,
                     help='learning rate')
  parser.add_argument('--decay_rate', type=float, default=1.0,
                     help='decay rate for rmsprop')
  parser.add_argument('--num_mixture', type=int, default=20,
                     help='number of gaussian mixtures')
  parser.add_argument('--data_scale', type=float, default=20,
                     help='factor to scale raw data down by')
  parser.add_argument('--mode', type=str, default='synthesis',
                     help='prediction / synthesis' )
  parser.add_argument('--load_model', type=str, default=None,
                     help='Reload a model checkpoint and restore training.' )
  parser.add_argument('--bptt_length', type=int, default=300,
                     help='How many steps should the gradients pass back.' )
  
  args = parser.parse_args(['--num_epochs','4'])

  train(args)

In [4]:

if __name__ == "__main__":
  main()


train data: 12187, valid data: 0
hyperparam. saved.
Sequence 1250/48748 (epoch 0), batch 102, train_loss = 1.692, time/(100*batch) = 38.107
Sequence 2450/48748 (epoch 0), batch 202, train_loss = 0.651, time/(100*batch) = 38.098
Sequence 3700/48748 (epoch 0), batch 304, train_loss = 0.505, time/(100*batch) = 37.955
Sequence 4900/48748 (epoch 0), batch 408, train_loss = 0.479, time/(100*batch) = 39.223
Sequence 6250/48748 (epoch 0), batch 511, train_loss = 0.798, time/(100*batch) = 37.924
Sequence 7500/48748 (epoch 0), batch 612, train_loss = 0.309, time/(100*batch) = 37.890
Sequence 8750/48748 (epoch 0), batch 715, train_loss = 0.409, time/(100*batch) = 37.845
Sequence 10000/48748 (epoch 0), batch 817, train_loss = 0.303, time/(100*batch) = 37.650
Sequence 11300/48748 (epoch 0), batch 919, train_loss = 0.519, time/(100*batch) = 37.166
Epoch 0 completed, average train loss 0.649594
Sequence 12587/48748 (epoch 1), batch 1022, train_loss = 0.187, time/(100*batch) = 36.691
Sequence 13837/48

### 2. Alternative training in Ipython

In [4]:
# Define arguments 
parser = argparse.ArgumentParser()
parser.add_argument('--dim_rec', type=int, default=400,
                     help='size of RNN hidden state')
parser.add_argument('--num_layers', type=int, default=3,
                     help='number of layers in the RNN. ' \
                     'Needs to be larger than 3 for synthesis.')
parser.add_argument('--batch_size', type=int, default=50,
                     help='minibatch size')
parser.add_argument('--num_epochs', type=int, default=200,
                     help='number of epochs')
parser.add_argument('--save_every', type=int, default=5,
                     help='save frequency by epoches')
parser.add_argument('--model_dir', type=str, default='checkpoints',
                     help='directory to save model to')
parser.add_argument('--summary_dir', type=str, default='summary',
                     help='directory to save tensorboard info')
parser.add_argument('--max_grad_norm', type=float, default=10.,
                     help='clip gradients at this value')
parser.add_argument('--learning_rate', type=float, default=0.001,
                     help='learning rate')
parser.add_argument('--decay_rate', type=float, default=1.0,
                     help='decay rate for rmsprop')
parser.add_argument('--num_mixture', type=int, default=20,
                     help='number of gaussian mixtures')
parser.add_argument('--data_scale', type=float, default=20,
                     help='factor to scale raw data down by')
parser.add_argument('--mode', type=str, default='synthesis',
                     help='prediction / synthesis' )
parser.add_argument('--load_model', type=str, default=None,
                     help='Reload a model checkpoint and restore training.' )
parser.add_argument('--bptt_length', type=int, default=300,
                     help='How many steps should the gradients pass back.' )
  
args = parser.parse_args(['--num_epochs','4'])


In [5]:
args.num_epochs = 4
args.save_every = 2
train(args)

train data: 12187, valid data: 0
checkpoints
hyperparam. saved.
data_loader.num_sequences: 12187, args.batch_size: 50
Sequence 1250/48748 (epoch 0), batch 102, train_loss = 1.940, time/(100*batch) = 39.823
Sequence 2450/48748 (epoch 0), batch 202, train_loss = 0.683, time/(100*batch) = 40.481
Sequence 3700/48748 (epoch 0), batch 304, train_loss = 0.605, time/(100*batch) = 39.665
Sequence 4900/48748 (epoch 0), batch 408, train_loss = 0.429, time/(100*batch) = 41.094
Sequence 6250/48748 (epoch 0), batch 511, train_loss = 0.787, time/(100*batch) = 39.597
Sequence 7500/48748 (epoch 0), batch 612, train_loss = 0.352, time/(100*batch) = 40.004
Sequence 8750/48748 (epoch 0), batch 715, train_loss = 0.446, time/(100*batch) = 39.391
Sequence 10000/48748 (epoch 0), batch 817, train_loss = 0.297, time/(100*batch) = 39.327
Sequence 11300/48748 (epoch 0), batch 919, train_loss = 0.475, time/(100*batch) = 38.534
x:  <class 'numpy.ndarray'> ; c <class 'numpy.ndarray'>
Epoch 0 completed, average train

In [10]:
def expand_and_repeat(data, axis, num_repeat):
    data = np.expand_dims(data, axis=axis)
    data = np.repeat(data, repeats=num_repeat, axis=axis)
    return data



max_char_len = 2
num_mixture = 3
batch_size = 5
char_index = np.arange(max_char_len, dtype=np.float32)
print('char_index1:',char_index)
char_index = expand_and_repeat(char_index, 0, num_mixture)
print('char_index2:',char_index)
char_index = expand_and_repeat(char_index, 0, batch_size)
print('char_index3:',char_index)
print(char_index.shape)

char_index1: [0. 1.]
char_index2: [[0. 1.]
 [0. 1.]
 [0. 1.]]
char_index3: [[[0. 1.]
  [0. 1.]
  [0. 1.]]

 [[0. 1.]
  [0. 1.]
  [0. 1.]]

 [[0. 1.]
  [0. 1.]
  [0. 1.]]

 [[0. 1.]
  [0. 1.]
  [0. 1.]]

 [[0. 1.]
  [0. 1.]
  [0. 1.]]]
(5, 3, 2)


In [11]:
with tf.variable_scope("foo"):
    v = tf.get_variable("v", [1])
with tf.variable_scope("foo", reuse=True):
    v1 = tf.get_variable("v", [1])
assert v1 == v

In [21]:
with tf.variable_scope("foo"):
    v = tf.get_variable("v", [1])
with tf.variable_scope("foo", reuse=True):
    v1 = tf.get_variable("v", [1])
assert v1 == v

ValueError: Variable foo/v already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-11-ef6e0cc39327>", line 2, in <module>
    v = tf.get_variable("v", [1])
  File "/home/mye/tools/anacondas3/envs/tensorflow/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/mye/tools/anacondas3/envs/tensorflow/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2821, in run_ast_nodes
    if self.run_code(code, result):
